In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ultimate-multi-organ-dataset4720/Ultimate_dataset/README.md
/kaggle/input/ultimate-multi-organ-dataset4720/Ultimate_dataset/ultimate.mat
/kaggle/input/ultimate-multi-organ-dataset4720/Ultimate_dataset/Training/img_folder/img4195.png
/kaggle/input/ultimate-multi-organ-dataset4720/Ultimate_dataset/Training/img_folder/img2945.png
/kaggle/input/ultimate-multi-organ-dataset4720/Ultimate_dataset/Training/img_folder/img3884.png
/kaggle/input/ultimate-multi-organ-dataset4720/Ultimate_dataset/Training/img_folder/img4643.png
/kaggle/input/ultimate-multi-organ-dataset4720/Ultimate_dataset/Training/img_folder/img4521.png
/kaggle/input/ultimate-multi-organ-dataset4720/Ultimate_dataset/Training/img_folder/img1748.png
/kaggle/input/ultimate-multi-organ-dataset4720/Ultimate_dataset/Training/img_folder/img2922.png
/kaggle/input/ultimate-multi-organ-dataset4720/Ultimate_dataset/Training/img_folder/img3398.png
/kaggle/input/ultimate-multi-organ-dataset4720/Ultimate_dataset/Training/img_fold

In [2]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model

def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape, num_classes):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(num_classes, 1, padding="same", activation="softmax")(d4)

    model = Model(inputs, outputs, name="U-Net")
    return model

if __name__ == "__main__":
    input_shape = (512, 512, 3)
    model = build_unet(input_shape,11)
    model.summary()

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Model: "U-Net"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 512, 512, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 512, 512, 64)         1792      ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 512, 512, 64)         256       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 512, 512, 64)         0         ['batch_normalization[0][0

In [3]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import pandas as pd
import cv2
from glob import glob
import scipy.io
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger

""" Global parameters """
global IMG_H
global IMG_W
global NUM_CLASSES
global CLASSES
global COLORMAP

""" Creating a directory """
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

""" Load and split the dataset """
def load_dataset(path, split=0.2):
    images = sorted(glob(os.path.join(path, "Training","img_folder", "*")))[:4720]
    masks = sorted(glob(os.path.join(path, "Training","mask_folder", "*")))[:4720]


    split_size = int(split * len(images))

    train_x, valid_x = train_test_split(images, test_size=split_size, random_state=42)
    train_y, valid_y = train_test_split(masks, test_size=split_size, random_state=42)

    train_x, test_x = train_test_split(train_x, test_size=split_size, random_state=42)
    train_y, test_y = train_test_split(train_y, test_size=split_size, random_state=42)

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)


def get_colormap(path):
    mat_path = "/kaggle/input/ultimate-multi-organ-dataset4720/Ultimate_dataset/ultimate.mat"
    colormap = scipy.io.loadmat(mat_path)["color"]

    classes = [
        "Background",
        "Spleen",
        "Right kidney",
        "Left kidney",
        "Gallbladder",
        "Liver",
        "Stomach",
        "Aorta",
        "Inferior vena cava",
        "Portal vein",
        "pancreas"
    ]

    return classes, colormap

def read_image(x):
    x = cv2.imread(x, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (IMG_W, IMG_H))
    x = x.astype(np.float32)
    return x

def read_mask(x):
    x = cv2.imread(x, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (IMG_W, IMG_H))

    #Masl processing
    output=[]
    # for i,color in enumerate(COLORMAP):
    #   cmap = np.all(np.equal(x, color), axis=-1)
    #   cv2.imwrite(f"cmap{i}.png", cmap*255)
    for color in COLORMAP:
      cmap = np.all(np.equal(x, color), axis=-1)
      output.append(cmap)


    output = np.stack(output, axis=-1)
    output = output.astype(np.uint8)

    return output


def preprocess(x, y):
    def f(x, y):
        x = x.decode()
        y = y.decode()

        x =read_image(x)
        y=read_mask(y)

        return x,y

    image, mask = tf.numpy_function(f, [x, y], [tf.float32, tf.uint8])
    image.set_shape([IMG_H, IMG_W, 3])
    mask.set_shape([IMG_H, IMG_W, NUM_CLASSES])

    return image, mask

def tf_dataset(x, y, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.shuffle(buffer_size=5000)
    dataset = dataset.map(preprocess)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(2)
    return dataset

if  __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Directory for storing files """
    create_dir("files")

    """ Hyperparameters """
    IMG_H = 320
    IMG_W = 416
    NUM_CLASSES = 11
    input_shape = (IMG_H, IMG_W,3)

    batch_size = 10
    lr = 1e-4
    num_epochs = 50
    dataset_path ="/kaggle/input/ultimate-multi-organ-dataset4720/Ultimate_dataset"
    model_path = os.path.join("/kaggle/working/files", "model4.h5")
    csv_path = os.path.join("/kaggle/working/files", "Abdomen_data.csv")

    load_dataset(dataset_path)

    """ Loading the dataset """
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_dataset(dataset_path)
    print(f"Train: {len(train_x)}/{len(train_y)} - Valid: {len(valid_x)}/{len(valid_y)} - Test: {len(test_x)}/{len(test_x)}")

    """ Process the colormap """
    CLASSES, COLORMAP = get_colormap(dataset_path)

    print("COLOR:",len(COLORMAP))

    # read_mask(train_y[10])

    """ Dataset Pipeline """
    train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
    valid_dataset = tf_dataset(valid_x, valid_y, batch=batch_size)

    """ Model """
    model = build_unet(input_shape, NUM_CLASSES)
    # model.load_weights(model_path)
    model.compile(
        loss="categorical_crossentropy",
        optimizer=tf.keras.optimizers.Adam(lr)
    )
    # model.summary()

    """ Training """
    callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
        CSVLogger(csv_path, append=True),
        EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False)
    ]

    model.fit(train_dataset,
        validation_data=valid_dataset,
        epochs=num_epochs,
        callbacks=callbacks
    )




Train: 2832/2832 - Valid: 944/944 - Test: 944/944
COLOR: 11
Epoch 1/50
284/284 [==============================] - ETA: 0s - loss: 0.6384
Epoch 1: val_loss improved from inf to 0.71485, saving model to /kaggle/working/files/model4.h5


/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


284/284 [==============================] - 328s 1s/step - loss: 0.6384 - val_loss: 0.7148 - lr: 1.0000e-04
Epoch 2/50
284/284 [==============================] - ETA: 0s - loss: 0.2832
Epoch 2: val_loss improved from 0.71485 to 0.26776, saving model to /kaggle/working/files/model4.h5
284/284 [==============================] - 258s 906ms/step - loss: 0.2832 - val_loss: 0.2678 - lr: 1.0000e-04
Epoch 3/50
284/284 [==============================] - ETA: 0s - loss: 0.1946
Epoch 3: val_loss improved from 0.26776 to 0.24066, saving model to /kaggle/working/files/model4.h5
284/284 [==============================] - 256s 899ms/step - loss: 0.1946 - val_loss: 0.2407 - lr: 1.0000e-04
Epoch 4/50
284/284 [==============================] - ETA: 0s - loss: 0.1435
Epoch 4: val_loss improved from 0.24066 to 0.14570, saving model to /kaggle/working/files/model4.h5
284/284 [==============================] - 256s 899ms/step - loss: 0.1435 - val_loss: 0.1457 - lr: 1.0000e-04
Epoch 5/50
284/284 [============


KeyboardInterrupt



In [8]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import numpy as np
import cv2
import pandas as pd
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score
import scipy.io
from sklearn.model_selection import train_test_split
import tensorflow as tf
from PIL import Image
# from train import load_dataset, create_dir, get_colormap

""" Global parameters """
global IMG_H
global IMG_W
global NUM_CLASSES
global CLASSES
global COLORMAP

""" Creating a directory """
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

""" Load and split the dataset """
def load_dataset(path, split=0.2):
    images = sorted(glob(os.path.join(path,"Training","img_folder", "*")))[:4720]
    masks = sorted(glob(os.path.join(path, "Training","mask_folder", "*")))[:4720]

    split_size = int(split * len(images))

    train_x, valid_x = train_test_split(images, test_size=split_size, random_state=42)
    train_y, valid_y = train_test_split(masks, test_size=split_size, random_state=42)

    train_x, test_x = train_test_split(train_x, test_size=split_size, random_state=42)
    train_y, test_y = train_test_split(train_y, test_size=split_size, random_state=42)

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)


def get_colormap(path):
    mat_path = os.path.join(path,"/kaggle/input/ultimate-multi-organ-dataset4720/Ultimate_dataset/ultimate.mat")
    colormap = scipy.io.loadmat(mat_path)["color"]

    classes = [
        "Background",
        "Spleen",
        "Right kidney",
        "Left kidney",
        "Gallbladder",
        "Liver",
        "Stomach",
        "Aorta",
        "Inferior vena cava",
        "Portal vein",
        "pancreas"
    ]

    return classes, colormap

def read_image(x):
    x = cv2.imread(x, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (IMG_W, IMG_H))
    x = x.astype(np.float32)
    return x

def read_mask(x):
    x = cv2.imread(x, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (IMG_W, IMG_H))

    #Masl processing
    output=[]
    # for i,color in enumerate(COLORMAP):
    #   cmap = np.all(np.equal(x, color), axis=-1)
    #   cv2.imwrite(f"cmap{i}.png", cmap*255)
    for color in COLORMAP:
      cmap = np.all(np.equal(x, color), axis=-1)
      output.append(cmap)


    output = np.stack(output, axis=-1)
    output = output.astype(np.uint8)

    return output


def preprocess(x, y):
    def f(x, y):
        x = x.decode()
        y = y.decode()

        x =read_image(x)
        y=read_mask(y)

        return x,y

    image, mask = tf.numpy_function(f, [x, y], [tf.float32, tf.uint8])
    image.set_shape([IMG_H, IMG_W, 3])
    mask.set_shape([IMG_H, IMG_W, NUM_CLASSES])

    return image, mask

def tf_dataset(x, y, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.shuffle(buffer_size=5000)
    dataset = dataset.map(preprocess)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(2)
    return dataset


def grayscale_to_rgb(mask, classes, colormap):
    h, w, _ = mask.shape
    mask = mask.astype(np.int32)
    output = []

    for i, pixel in enumerate(mask.flatten()):
        output.append(colormap[pixel])

    output = np.reshape(output, (h, w, 3))
    return output

def save_results(image, mask, pred, save_image_path):
    # print(image.shape,mask.shape,pred.shape)
    h, w, _ = image.shape
    line = np.ones((h, 10, 3)) * 255

    pred = np.expand_dims(pred, axis=-1)
    pred = grayscale_to_rgb(pred, CLASSES, COLORMAP)

    # Ensure both images have the same shape
    assert image.shape == mask.shape, "Images must have the same shape"
    assert image.shape == pred.shape

    # Blend the images using the alpha parameter
    alpha = 0.5
    blended_image1 = alpha * image + (1 - alpha) * mask
    blended_image2 = alpha * image + (1 - alpha) * pred
    # alpha = 0.5
    # blended_image1 = Image.blend(image, mask, alpha)
    # blended_image2 = Image.blend(image, pred, alpha)

    cat_images = np.concatenate([image, line, mask, line, pred, line, blended_image1, blended_image2], axis=1)
    cv2.imwrite(save_image_path, cat_images)


if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Directory for storing files """
    create_dir("results2/predictions")

    """ Hyperparameters """
    IMG_H = 320
    IMG_W = 416
    NUM_CLASSES = 11
    dataset_path = "/kaggle/input/ultimate-multi-organ-dataset4720/Ultimate_dataset"
    # dataset_path = "/content/drive/MyDrive/human"
    # model_path = os.path.join("files", "model1.h5")
    model_path = "/kaggle/working/files/model4.h5"

    """ Colormap """
    CLASSES, COLORMAP = get_colormap(dataset_path)

    """ Model """
    model = tf.keras.models.load_model(model_path)
    # model.summary()

    """ Load the dataset """
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_dataset(dataset_path)
    print(f"Train: {len(train_x)}/{len(train_y)} - Valid: {len(valid_x)}/{len(valid_y)} - Test: {len(test_x)}/{len(test_y)}")
    print("")

    # Prediction and Evaluation

    SCORE = []
    for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
      name = x.split("/")[-1].split(".")[0]

      """ Reading the image """
      image = cv2.imread(x, cv2.IMREAD_COLOR)
      image = cv2.resize(image, (IMG_W, IMG_H))
      image_x = image
      # image = image/255.0
      image = np.expand_dims(image, axis=0)

      """ Reading the mask """
      mask = cv2.imread(y, cv2.IMREAD_COLOR)
      mask = cv2.resize(mask, (IMG_W, IMG_H))
      mask_x = mask
      onehot_mask = []
      for color in COLORMAP:
          cmap = np.all(np.equal(mask, color), axis=-1)
          onehot_mask.append(cmap)
      onehot_mask = np.stack(onehot_mask, axis=-1)
      onehot_mask = np.argmax(onehot_mask, axis=-1)
      onehot_mask = onehot_mask.astype(np.int32)

      """ Prediction """
      pred = model.predict(image, verbose=0)[0]
      pred = np.argmax(pred, axis=-1)
      pred = pred.astype(np.float32)

      """ Saving the prediction """
      save_image_path = f"results2/predictions/{name}.png"
      save_results(image_x, mask_x, pred, save_image_path)

      """ Flatten the array """
      onehot_mask = onehot_mask.flatten()
      pred = pred.flatten()

      labels = [i for i in range(NUM_CLASSES)]

      """ Calculating the metrics values """
      f1_value = f1_score(onehot_mask, pred, labels=labels, average=None, zero_division=0)
      jac_value = jaccard_score(onehot_mask, pred, labels=labels, average=None, zero_division=0)

      SCORE.append([f1_value, jac_value])


    # """ Metrics values """
    # score = np.array(SCORE)
    # score = np.mean(score, axis=0)

    # f = open("files/score.csv", "w")
    # f.write("Class,F1,Jaccard\n")

    # l = ["Class", "F1", "Jaccard"]
    # print(f"{l[0]:15s} {l[1]:10s} {l[2]:10s}")
    # print("-"*35)

    # for i in range(score.shape[1]):
    #     class_name = CLASSES[i]
    #     f1 = score[0, i]
    #     jac = score[1, i]
    #     dstr = f"{class_name:15s}: {f1:1.5f} - {jac:1.5f}"
    #     print(dstr)
    #     f.write(f"{class_name:15s},{f1:1.5f},{jac:1.5f}\n")

    # print("-"*35)
    # class_mean = np.mean(score, axis=-1)
    # class_name = "Mean"
    # f1 = class_mean[0]
    # jac = class_mean[1]
    # dstr = f"{class_name:15s}: {f1:1.5f} - {jac:1.5f}"
    # print(dstr)
    # f.write(f"{class_name:15s},{f1:1.5f},{jac:1.5f}\n")

    # f.close()

    """ Metrics values """
    score = np.array(SCORE)
    score = np.mean(score, axis=0)

    # Calculate accuracy using true labels and predicted labels
    accuracy = accuracy_score(onehot_mask, pred.flatten())

    f = open("results2/score.csv", "w")
    f.write("Class,F1,Jaccard,Accuracy\n")

    l = ["Class", "F1", "Jaccard", "Accuracy"]
    print(f"{l[0]:15s} {l[1]:10s} {l[2]:10s} {l[3]:10s}")
    print("-" * 50)

    for i in range(score.shape[1]):
        class_name = CLASSES[i]
        f1 = score[0, i]
        jac = score[1, i]
        dstr = f"{class_name:15s}: {f1:1.5f} - {jac:1.5f} - {accuracy:1.5f}"
        print(dstr)
        f.write(f"{class_name:15s},{f1:1.5f},{jac:1.5f},{accuracy:1.5f}\n")

    print("-" * 50)
    class_mean = np.mean(score, axis=-1)
    class_name = "Mean"
    f1 = class_mean[0]
    jac = class_mean[1]
    dstr = f"{class_name:15s}: {f1:1.5f} - {jac:1.5f} - {accuracy:1.5f}"
    print(dstr)
    f.write(f"{class_name:15s},{f1:1.5f},{jac:1.5f},{accuracy:1.5f}\n")
    f.close()

Train: 2832/2832 - Valid: 944/944 - Test: 944/944



100%|██████████| 944/944 [06:50<00:00,  2.30it/s]

Class           F1         Jaccard    Accuracy  
--------------------------------------------------
Background     : 0.99526 - 0.99060 - 0.98398
Spleen         : 0.39129 - 0.36519 - 0.98398
Right kidney   : 0.44366 - 0.40816 - 0.98398
Left kidney    : 0.44439 - 0.41133 - 0.98398
Gallbladder    : 0.74071 - 0.70495 - 0.98398
Liver          : 0.05559 - 0.04173 - 0.98398
Stomach        : 0.46757 - 0.42204 - 0.98398
Aorta          : 0.79050 - 0.72546 - 0.98398
Inferior vena cava: 0.74649 - 0.66192 - 0.98398
Portal vein    : 0.15478 - 0.11597 - 0.98398
pancreas       : 0.32679 - 0.26923 - 0.98398
--------------------------------------------------
Mean           : 0.50519 - 0.46514 - 0.98398


In [11]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import numpy as np
import cv2
import pandas as pd
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score
import scipy.io
from sklearn.model_selection import train_test_split
import tensorflow as tf
from PIL import Image
# from train import load_dataset, create_dir, get_colormap

""" Global parameters """
global IMG_H
global IMG_W
global NUM_CLASSES
global CLASSES
global COLORMAP

""" Creating a directory """
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

""" Load and split the dataset """
def load_dataset(path, split=0.2):
    images = sorted(glob(os.path.join(path, "Training", "img_folder", "*")))[:4720]
    masks = sorted(glob(os.path.join(path, "Training", "mask_folder", "*")))[:4720]

    split_size = int(split * len(images))

    train_x, valid_x = train_test_split(images, test_size=split_size, random_state=42)
    train_y, valid_y = train_test_split(masks, test_size=split_size, random_state=42)

    train_x, test_x = train_test_split(train_x, test_size=split_size, random_state=42)
    train_y, test_y = train_test_split(train_y, test_size=split_size, random_state=42)

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)


def get_colormap(path):
    mat_path = os.path.join(path, "ultimate.mat")
    colormap = scipy.io.loadmat(mat_path)["color"]

    classes = [
        "Background",
        "Spleen",
        "Right kidney",
        "Left kidney",
        "Gallbladder",
        "Liver",
        "Stomach",
        "Aorta",
        "Inferior vena cava",
        "Portal vein",
        "Pancreas"
    ]

    return classes, colormap

def read_image(x):
    x = cv2.imread(x, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (IMG_W, IMG_H))
    x = x.astype(np.float32)
    return x

def read_mask(x):
    x = cv2.imread(x, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (IMG_W, IMG_H))

    #Masl processing
    output=[]
    # for i,color in enumerate(COLORMAP):
    #   cmap = np.all(np.equal(x, color), axis=-1)
    #   cv2.imwrite(f"cmap{i}.png", cmap*255)
    for color in COLORMAP:
      cmap = np.all(np.equal(x, color), axis=-1)
      output.append(cmap)


    output = np.stack(output, axis=-1)
    output = output.astype(np.uint8)

    return output


def preprocess(x, y):
    def f(x, y):
        x = x.decode()
        y = y.decode()

        x =read_image(x)
        y=read_mask(y)

        return x,y

    image, mask = tf.numpy_function(f, [x, y], [tf.float32, tf.uint8])
    image.set_shape([IMG_H, IMG_W, 3])
    mask.set_shape([IMG_H, IMG_W, NUM_CLASSES])

    return image, mask

def tf_dataset(x, y, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.shuffle(buffer_size=5000)
    dataset = dataset.map(preprocess)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(2)
    return dataset


def grayscale_to_rgb(mask, classes, colormap):
    h, w, _ = mask.shape
    mask = mask.astype(np.int32)
    output = []

    for i, pixel in enumerate(mask.flatten()):
        output.append(colormap[pixel])

    output = np.reshape(output, (h, w, 3))
    return output

def save_results(image, mask, pred, save_image_path):
    # print(image.shape,mask.shape,pred.shape)
    h, w, _ = image.shape
    line = np.ones((h, 10, 3)) * 255

    pred = np.expand_dims(pred, axis=-1)
    pred = grayscale_to_rgb(pred, CLASSES, COLORMAP)

    # Ensure both images have the same shape
    assert image.shape == mask.shape
    assert image.shape == pred.shape

    # Blend the images using the alpha parameter
    alpha = 0.5
    blended_image1 = alpha * image + (1 - alpha) * mask
    blended_image2 = alpha * image + (1 - alpha) * pred
    # alpha = 0.5
    # blended_image1 = Image.blend(image, mask, alpha)
    # blended_image2 = Image.blend(image, pred, alpha)

    cat_images = np.concatenate([image, line, mask, line, pred, line, blended_image1, blended_image2], axis=1)
    cv2.imwrite(save_image_path, cat_images)


if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Directory for storing files """
    create_dir("files")

    """ Directory for storing files """
    create_dir("results2/predictions")

    """ Hyperparameters """
    IMG_H = 256
    IMG_W = 256
    NUM_CLASSES = 11
    dataset_path = "/kaggle/input/ultimate-multi-organ-dataset4720/Ultimate_dataset"
    # dataset_path = "/content/drive/MyDrive/human"
    # model_path = os.path.join("files", "model1.h5")
    model_path = ""

    """ Colormap """
    CLASSES, COLORMAP = get_colormap(dataset_path)

    """ Model """
    model = tf.keras.models.load_model(model_path)
    # model.summary()

    """ Load the dataset """
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_dataset(dataset_path)
    print(f"Train: {len(train_x)}/{len(train_y)} - Valid: {len(valid_x)}/{len(valid_y)} - Test: {len(test_x)}/{len(test_y)}")
    print("")

    # Prediction and Evaluation

    SCORE = []
    for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
      name = x.split("/")[-1].split(".")[0]

      """ Reading the image """
      image = cv2.imread(x, cv2.IMREAD_COLOR)
      image = cv2.resize(image, (IMG_W, IMG_H))
      image_x = image
      # image = image/255.0
      image = np.expand_dims(image, axis=0)

      """ Reading the mask """
      mask = cv2.imread(y, cv2.IMREAD_COLOR)
      mask = cv2.resize(mask, (IMG_W, IMG_H))
      mask_x = mask
      onehot_mask = []
      for color in COLORMAP:
          cmap = np.all(np.equal(mask, color), axis=-1)
          onehot_mask.append(cmap)
      onehot_mask = np.stack(onehot_mask, axis=-1)
      onehot_mask = np.argmax(onehot_mask, axis=-1)
      onehot_mask = onehot_mask.astype(np.int32)

      """ Prediction """
      pred = model.predict(image, verbose=0)[0]
      pred = np.argmax(pred, axis=-1)
      pred = pred.astype(np.float32)

      """ Saving the prediction """
      save_image_path = f"results2/predictions/{name}.png"
      save_results(image_x, mask_x, pred, save_image_path)

      """ Flatten the array """
      onehot_mask = onehot_mask.flatten()
      pred = pred.flatten()

      labels = [i for i in range(NUM_CLASSES)]

      """ Calculating the metrics values """
      f1_value = f1_score(onehot_mask, pred, labels=labels, average=None, zero_division=0)
      jac_value = jaccard_score(onehot_mask, pred, labels=labels, average=None, zero_division=0)
    
      SCORE.append([f1_value, jac_value])

    """ Metrics values """
    score = np.array(SCORE)
    score = np.mean(score, axis=0)

    # Calculate accuracy using true labels and predicted labels
    accuracy = accuracy_score(onehot_mask, pred.flatten())

    f = open("files/score.csv", "w")
    f.write("Class,F1,Jaccard,Accuracy\n")

    l = ["Class", "F1", "Jaccard", "Accuracy"]
    print(f"{l[0]:15s} {l[1]:10s} {l[2]:10s} {l[3]:10s}")
    print("-" * 50)

    for i in range(score.shape[1]):
        class_name = CLASSES[i]
        f1 = score[0, i]
        jac = score[1, i]
        dstr = f"{class_name:15s}: {f1:1.5f} - {jac:1.5f} - {accuracy:1.5f}"
        print(dstr)
        f.write(f"{class_name:15s},{f1:1.5f},{jac:1.5f},{accuracy:1.5f}\n")
     
    print("-" * 50)
    class_mean = np.mean(score, axis=-1)
    class_name = "Mean"
    f1 = class_mean[0]
    jac = class_mean[1]
    dstr = f"{class_name:15s}: {f1:1.5f} - {jac:1.5f} - {accuracy:1.5f}"
    print(dstr)
    f.write(f"{class_name:15s},{f1:1.5f},{jac:1.5f},{accuracy:1.5f}\n")

    f.close()

Train: 2832/2832 - Valid: 944/944 - Test: 944/944



  0%|          | 0/944 [00:00<?, ?it/s]


ValueError: in user code:

    File "/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py", line 2341, in predict_function  *
        return step_function(self, iterator)
    File "/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py", line 2327, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py", line 2315, in run_step  **
        outputs = model.predict_step(data)
    File "/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py", line 2283, in predict_step
        return self(x, training=False)
    File "/opt/conda/lib/python3.10/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/opt/conda/lib/python3.10/site-packages/keras/src/engine/input_spec.py", line 298, in assert_input_compatibility
        raise ValueError(

    ValueErro

In [12]:
import shutil

# Source folder containing images
source_folder = '/kaggle/working/results2/predictions'

# Destination zip file
zip_filename = '/kaggle/working/predictions.zip'

# Create a zip file
shutil.make_archive(zip_filename.replace('.zip', ''), 'zip', source_folder)


'/kaggle/working/predictions.zip'

In [13]:
import os

# Specify the path to the directory
directory_path = '/kaggle/working/results2/predictions'

# Get the list of files in the directory
files = os.listdir(directory_path)

# Filter only files (not directories) if needed
files = [file for file in files if os.path.isfile(os.path.join(directory_path, file))]

# Print the number of files
print(f"Number of files in the directory: {len(files)}")


Number of files in the directory: 944
